In [7]:
from kosmoss import CONFIG, PROCESSED_DATA_PATH
import os.path as osp
import shutil

In [11]:
_, used_space, _ = shutil.disk_usage(osp.join(PROCESSED_DATA_PATH, f"graphs-{CONFIG['timestep']}"))
used_space // (2**30)

187

In [11]:
psutil.disk_usage(osp.join(PROCESSED_DATA_PATH, f"graphs-{CONFIG['timestep']}"))

sdiskusage(total=1055815524352, used=201310593024, free=854488154112, percent=19.1)

In [1]:
926 // 100

9

In [2]:
926 % 100

26

In [3]:
1085440 / 6784

160.0

In [4]:
1084000 // 160

6775

In [5]:
922309 // (1085440 // 6784)

5764

In [1]:
kwargs = {
    "logger": "hi",
    "gpus": "hfdis",
    "strategy": "ddp",
    "progress_bar_refresh_rate": 0
}

In [4]:
list(kwargs.keys())

['logger', 'gpus', 'strategy', 'progress_bar_refresh_rate']

In [5]:
import pytorch_lightning as pl
pl.__version__

'1.5.10'

In [6]:
import ray
ray.__version__

'1.10.0'